# Microscopic traffic simulation using `F#`
------
![:height="36px" width="36px"](https://cdn-images-1.medium.com/max/1500/1*pBXdKxuvv5h-zBTioDE1PQ.jpeg)

F# is a pragmatic, functional first, multi-paradigm language. It shines in modelling real life situations. Therefore it is a very good choice when doing simulations.

This notebook presents an approach to do microscoping trafffic simulation based on this paper https://nptel.ac.in/courses/105101008/downloads/cete_16.pdf 



The generation of vehicles using negative exponential distribution is demonstrated here. The probability distribution function is given as follows.

## $$f(x) = \lambda e^{-\lambda x}$$
The following equation is reprsented by the function `f(x)` in the next cell

In [ ]:
let lambda = 15. //λ = 900/60 = 15 vehicle/min.
let f x = lambda * exp(-lambda * x)

In [ ]:
open System
let mu = 4.//μ = 1(900/3600) = 4sec

Calculate the headways and then estimate the cumulative headways. The calculations are given by the following equation
Where `R` is a random number between 0 and 1

## $$X = \mu (-log_e R)$$

In [ ]:
//This is how we can generate X
let X  = mu * (- log ((new Random()).NextDouble()))

In a similar way, the vehicular arrival pattern can be modeled using Poisson’s distribution. The probability mass function is given as:

## $$p(x)=\frac{\lambda^{x}e^{-\lambda}}{x!}$$

The following recursive function defines the factorial calculation for a given integer. Note that it uses pattern matching feature of F#

In [5]:
let rec factorial x  = 
   match x with 
   | 0 -> 1
   | 1 -> 1
   | _ -> x * factorial (x - 1) 


In [6]:
//Poisson's distribution is defined by the following function
let p x = 
    let numerator = (exp(-lambda) * (lambda ** (float)x))
    let denominator = factorial x
    numerator/float denominator

In [ ]:
p 5

In [8]:
p 1

4.588534808e-06

In [ ]:
p -1

In [ ]:
p 2

The following formula parameterizes calculation of Headway (`X`)

In [ ]:
let X_ (R:double) = mu * (- log R) 

In [ ]:
X_ 0.73 //An example call for a random number (See page # 7)

In [ ]:
//Generating Random numbers efficiently (prior to generate the Microscopic Traffic Simulation)
let randoms : System.Collections.Generic.List<float>   = new System.Collections.Generic.List<float>()
for i in [1 .. 40] do 
    randoms.Add((new Random()).NextDouble())
    System.Threading.Thread.Sleep(100)
printfn("Done")

In [ ]:
randoms

In [ ]:
#load "XPlot.Plotly.fsx"

In [ ]:
open XPlot.Plotly

In [ ]:
let data = randoms |> Seq.map (fun r -> (r, X_ r)) //Calculating the data to be used for checking the model of the simulation

In [ ]:
data

In [ ]:
//Attempting to plot the negagive distribution (that is being used to simulate vehicle arrival)
data |> Chart.Column |> Chart.WithTitle "Calculated Headways"

Calculating arrival times for all generated/predicted vehicles arrival. 

In [ ]:
let arrivalTimes = new System.Collections.Generic.List<float>()
for x in data do 
    arrivalTimes.Add(snd x)

In [ ]:
arrivalTimes

Cumulative arrival times for all the vehicles are calculated below

In [ ]:
let mutable foldedTime = 0.0
let arrivals = new System.Collections.Generic.List<float>()
for t in arrivalTimes do 
   foldedTime <- foldedTime + t
   arrivals.Add(foldedTime)
arrivals

In [ ]:
arrivals

In [ ]:
arrivals |> Chart.Column |> Chart.WithTitle "Simulated arrival time for vehicles" 
         

In [ ]:
//Type to represent a simulated row in 
type SimRow = {VehicleNumber: int; R: float; X : float; ArrivalTime:float}

In [ ]:
//The simulation as a projection of sequence of type "SimRow"
randoms |> Seq.mapi (fun index r -> {VehicleNumber = index; R = r; X = (X_ r); ArrivalTime = arrivalTimes.[index] })

### Vehicle arrivals using Negative exponential distribution

In [ ]:
//Wrapping the generation of simulated rows in a function 
let getSimRows(nums:System.Collections.Generic.List<float>)=
   let mutable cumulativeTime = 0.0
   let arrivalTimes = new System.Collections.Generic.List<float>()
   for i in 0 .. nums.Count-1 do 
       let part = X_ nums.[i]
       cumulativeTime <- cumulativeTime  + part
       arrivalTimes.Add(cumulativeTime)
  
   nums|> Seq.mapi (fun index r -> {VehicleNumber = index; R = r; X = (X_ r); ArrivalTime = arrivalTimes.[index] })
   

In [ ]:
printfn "%A" (getSimRows randoms)

In [ ]:
printfn "Vehicle Number                 | R            |X             |ArrivalTime(sec)"
let results = List.ofSeq( getSimRows randoms)
for i in 0 .. randoms.Count-1 do 
    printfn "%s                             | %f     |%f      |%f" ((results.[i].VehicleNumber + 1).ToString("00")) 
                                             (results.[i].R)  (results.[i].X)  (results.[i].ArrivalTime) 

## Simulated Model Calibration

The activity of specifying data to the model that describes traffic operations and other features
which are site specific is called calibration of the model. In other words, calibration is the
process of quantifying model parameters using real-world data. This data may take the form
of scalar elements and of statistical distributions. Calibration is a major challenge during the
implementation stage of any model. The commonly used methods of calibration are regression,
optimization, error determination, trajectory analysis etc. A brief description about various
errors and their significance is presented in this section

The optimization method of calibration is also explained using the following example problem.

### Numerical Example
The parameters obtained in GM car-following model simulation are given in Table below. Field
observed values of acceleration of follower is also given. Calibrate the model by finding the value
of α. Assume l=1 and m=0. Use optimization method to solve the problem.

Solution 
Step 1: Formulate the objective function (z).
Minimize 

## $$z = \sum_{i=1}^4({a_i}^{obs} - {a_i}^{cal})^2$$


| Observed Acceleration (aobs)        | Velocity difference dv           | Distance headway dx  |
| ------------- |:-------------:| -----:|
0.23| 1.5| 29.13
0.46| 1.88| 29.97
0.67| 1.16| 30.73
0.82| 0.32| 31.10



### Root Mean Square Error

## $$RMSE = \sqrt{\frac{1}{N}\sum_{i=1}N(x_i-y_i)^2 }$$

In [ ]:
let rmse (xs : float list) (ys:float list) = 
    let N =  xs |> List.length |> float
    let squared_diff = List.zip xs ys 
                      |> List.map (fun pair -> float(N*(fst pair - snd pair)) ** 2.) 
                      |> List.sum
    sqrt(squared_diff/(float N))

## $$RMSNE = \sqrt{\frac{1}{N}\sum_{i=1}N(\frac{x_i-y_i}{y_i})^2}$$

### Mean Error 

## $$ME = {\frac{1}{N}\sum_{i=1}N(x_i-y_i)}$$

In [ ]:
let mean_error (xs:float list) (ys:float list) = 
  let N = float xs.Length
  let num  = List.zip xs ys 
                 |> List.map (fun pair -> N * (fst pair - snd pair)) 
                 |> List.sum 
  float num/float xs.Length 

mean_error [1.3;2.2;4.24;5.2][1.2;4.2;5.6;2.1]

## Mean Normalized Error

## $$MNE = {\frac{1}{N}\sum_{i=1}N(\frac{x_i-y_i}{y_i})}$$

The above error measures are useful when applied separately to measurements at each location instead of to all measurements jointly. They indicate the existence of systematic bias in terms of under or over prediction by the simulation model. Taking into account that the series of measurements and simulated values can be collected at regular time intervals, it becomes obvious that they can be interpreted as time series and, therefore, used to determine how close the simulated and the observed values are. Thus it can be determined that how similar both time series are. On the other hand, the use of aggregated values to validate a simulation seems contradictory if one takes into account that it is dynamic in nature, and thus time dependent. Theil deﬁned a set of indices aimed at this goal and these indices have been widely used for that purpose. The ﬁrst index is Theil’s indicator, U (also called Theil’s inequality coeﬃcient), which provides a normalized measure of the relative error that reduces the impact of large errors: 


The global index U is bounded, 0 ≤ U ≤ 1, with U = 0 for a perfect ﬁt and xi = yi for i = 1 to N, between observed and simulated values. For U ≤ 0.2, the simulated series can be accepted as replicating the observed series acceptably well. The closer the values are to 0, the better will be the model. For values greater than 0.2, the simulated series is rejected.


## Theil's indicator ( a.k.a Theil's inequality coefficient)
### $$ U = \frac{\sqrt{\frac{1}{N}\sum_{i=1}N(x_i-y_i)^2}}{\sqrt{\frac{1}{N}\sum_{i=1}N(x_i)^2} +\sqrt{\frac{1}{N}\sum_{i=1}N(y_i)^2} }$$

In [ ]:

let theils (xs:float list) (ys:float list) = 
    let N = float xs.Length 
    let sqr x = x * x // Local function
    let numerator = rmse xs ys 
    
    //Breaking down the denominator into left and right
    let left = xs |> List.map (fun t -> N* t**2.) |> List.average
    let right = ys |> List.map (fun t -> N* t**2.) |> List.average 
    
    let denominator  = sqrt left + sqrt right 
    numerator/denominator

In [ ]:
theils [1.3;4.3;0.4;5.2] [2.3;4.3;0.4;5.1]